# Working with llama.cpp in Pixeltable

This tutorial demonstrates how to use Pixeltable's built-in `llama.cpp` integration to run local LLMs efficiently.

<div class="alert alert-block alert-info"><!-- mdx:none -->
<b>If you are running this tutorial in Colab:</b>
In order to make the tutorial run a bit snappier, let's switch to a GPU-equipped instance for this Colab session. To do that, click on the <code>Runtime -> Change runtime type</code> menu item at the top, then select the <code>GPU</code> radio button and click on <code>Save</code>.
</div>

### Important notes

- Models are automatically downloaded from Hugging Face and cached locally
- Different quantization levels are available for performance/quality tradeoffs
- Consider memory usage when choosing models and quantizations

## Set up environment

First, let's install Pixeltable with llama.cpp support:

In [ ]:
%pip install -qU pixeltable llama-cpp-python huggingface-hub

## Create a table for chat completions

Now let's create a table that will contain our inputs and responses.

In [1]:
import pixeltable as pxt
from pixeltable.functions import llama_cpp

pxt.drop_dir('llama_demo', force=True)
pxt.create_dir('llama_demo')

t = pxt.create_table('llama_demo/chat', {'input': pxt.String})

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/asiegel/.pixeltable/pgdata
Created directory 'llama_demo'.
Created table 'chat'.


Next, we add a computed column that calls the Pixeltable `create_chat_completion` UDF, which adapts the corresponding llama.cpp API call. In our examples, we'll use pretrained models from the Hugging Face repository. llama.cpp makes it easy to do this by specifying a repo_id (from the URL of the model) and filename from the model repo; the model will then be downloaded and cached automatically.

(If this is your first time using Pixeltable, the <a href="https://docs.pixeltable.com/tutorials/tables-and-data-operations">Pixeltable Fundamentals</a> tutorial contains more details about table creation, computed columns, and UDFs.)

For this demo we'll use `Qwen2.5-0.5B`, a very small (0.5-billion parameter) model that still produces decent results. We'll use `Q5_K_M` (5-bit) quantization, which gives an excellent balance of quality and efficiency.

In [2]:
# Add a computed column that uses llama.cpp for chat completion
# against the input.

messages = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': t.input}
]

t.add_computed_column(result=llama_cpp.create_chat_completion(
    messages,
    repo_id='Qwen/Qwen2.5-0.5B-Instruct-GGUF',
    repo_filename='*q5_k_m.gguf'
))

# Extract the output content from the JSON structure returned
# by llama_cpp.

t.add_computed_column(output=t.result.choices[0].message.content)

Added 0 column values with 0 errors in 0.01 s
Added 0 column values with 0 errors in 0.01 s


No rows affected.

## Test chat completion

Let's try a simple query:

In [3]:
# Test with a simple question
t.insert([
    {'input': 'What is the capital of France?'},
    {'input': 'What are some edible species of fish?'},
    {'input': 'Who are the most prominent classical composers?'}
])

llama_context: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64  

Inserted 3 rows with 0 errors in 6.74 s (0.44 rows/s)


3 rows inserted.

In [4]:
t.select(t.input, t.output).collect()

input,output
What is the capital of France?,The capital of France is Paris.
What are some edible species of fish?,"There are many edible species of fish, including: 1. Tilapia - a popular fish for both freshwater and saltwater environments. It's known for its small size and easy to cook. 2. Cod - a large fish that can grow to be over 20 pounds. It's often used in fish stew or as a main course. 3. Salmon - a fish that can be found in both freshwater and saltwater environments. It's known for its rich, flavorful flesh and is often used in fish tacos or as a main course. 4. Trout - a fish that can be fo ...... slightly sweet flavor and is often used in fish tacos or as a main course. 8. Flounder - a fish that can be found in both freshwater and saltwater environments. It's known for its mild, slightly sweet flavor and is often used in fish tacos or as a main course. 9. Tilapia - a popular fish for both freshwater and saltwater environments. It's known for its small size and easy to cook. 10. Cod - a large fish that can grow to be over 20 pounds. It's often used in fish stew or as a main course."
Who are the most prominent classical composers?,"The most prominent classical composers include: 1. Ludwig van Beethoven - known for his symphonies, piano sonatas, and operas 2. Wolfgang Amadeus Mozart - known for his operas, symphonies, and chamber music 3. Johann Sebastian Bach - known for his sacred works, concertos, and fugues 4. Antonio Vivaldi - known for his concertos, string quartets, and violin sonatas 5. Ludwig van Beethoven - known for his symphonies, piano sonatas, and operas 6. George Frideric Handel - known for his oratorios, operas, and string quartets 7. Johann Sebastian Bach - known for his sacred works, concertos, and fugues 8. Antonio Vivaldi - known for his concertos, string quartets, and violin sonatas 9. George Frideric Handel - known for his oratorios, operas, and string quartets 10. Johann Sebastian Bach - known for his sacred works, concertos, and fugues These composers have had a significant impact on the development of classical music and continue to be studied and performed to this day."


## Comparing models

Local model frameworks like `llama.cpp` make it easy to compare the output of different models. Let's try comparing the output from `Qwen` against a somewhat larger model, `Llama-3.2-1B`. As always, when we add a new computed column to our table, it's automatically evaluated against the existing table rows.

In [5]:
t.add_computed_column(result_l3=llama_cpp.create_chat_completion(
    messages,
    repo_id='bartowski/Llama-3.2-1B-Instruct-GGUF',
    repo_filename='*Q5_K_M.gguf'
))

t.add_computed_column(output_l3=t.result_l3.choices[0].message.content)

t.select(t.input, t.output, t.output_l3).collect()

llama_context: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

Added 3 column values with 0 errors in 6.32 s (0.47 rows/s)
Added 3 column values with 0 errors in 0.03 s (113.79 rows/s)


input,output,output_l3
What is the capital of France?,The capital of France is Paris.,The capital of France is Paris.
What are some edible species of fish?,"There are many edible species of fish, including: 1. Tilapia - a popular fish for both freshwater and saltwater environments. It's known for its small size and easy to cook. 2. Cod - a large fish that can grow to be over 20 pounds. It's often used in fish stew or as a main course. 3. Salmon - a fish that can be found in both freshwater and saltwater environments. It's known for its rich, flavorful flesh and is often used in fish tacos or as a main course. 4. Trout - a fish that can be fo ...... slightly sweet flavor and is often used in fish tacos or as a main course. 8. Flounder - a fish that can be found in both freshwater and saltwater environments. It's known for its mild, slightly sweet flavor and is often used in fish tacos or as a main course. 9. Tilapia - a popular fish for both freshwater and saltwater environments. It's known for its small size and easy to cook. 10. Cod - a large fish that can grow to be over 20 pounds. It's often used in fish stew or as a main course.","There are many edible species of fish, and the list can vary depending on the region and personal preferences. Here are some popular and commonly consumed edible fish species: 1. Salmon (Salmo salar): Native to the Pacific coast of North America, salmon is a fatty fish rich in omega-3 fatty acids and protein. 2. Tuna (Thunnus thynnus): A popular species of fish, tuna is found in tropical and temperate waters and is high in protein and low in fat. 3. Sardines (Sardinops caeruleus): These s ...... icus): A mild-flavored fish, tilapia is widely farmed and consumed in many parts of the world. 10. Shrimp (Penaeus spp.): A popular seafood choice, shrimp is often cooked in a variety of ways and is a good source of protein. Some other edible fish species worth mentioning include: - Eel (Anguilla anguilla) - Perch (Perca fluviatilis) - Trout (Oncorhynchus mykiss) - Bass (Micropterus salmoides) - Herring (Clupea harengus) Remember to always check local regulations and ensure that the fish"
Who are the most prominent classical composers?,"The most prominent classical composers include: 1. Ludwig van Beethoven - known for his symphonies, piano sonatas, and operas 2. Wolfgang Amadeus Mozart - known for his operas, symphonies, and chamber music 3. Johann Sebastian Bach - known for his sacred works, concertos, and fugues 4. Antonio Vivaldi - known for his concertos, string quartets, and violin sonatas 5. Ludwig van Beethoven - known for his symphonies, piano sonatas, and operas 6. George Frideric Handel - known for his oratorios, operas, and string quartets 7. Johann Sebastian Bach - known for his sacred works, concertos, and fugues 8. Antonio Vivaldi - known for his concertos, string quartets, and violin sonatas 9. George Frideric Handel - known for his oratorios, operas, and string quartets 10. Johann Sebastian Bach - known for his sacred works, concertos, and fugues These composers have had a significant impact on the development of classical music and continue to be studied and performed to this day.","Here's a list of some of the most prominent classical composers: **Classical Period (1750-1820)** 1. Wolfgang Amadeus Mozart (1756-1791) - Austrian composer known for his melodic genius and operas like 'The Magic Flute.' 2. Ludwig van Beethoven (1770-1827) - German composer who revolutionized classical music with his symphonies and piano sonatas. 3. Antonio Vivaldi (1678-1741) - Italian composer and violinist famous for his concertos, particularly 'The Four Seasons.' 4. Christoph Willibald ...... (1797-1828) - Austrian composer known for his melodic genius and symphonies like 'Unfinished Symphony.' **Romantic Period (1820-1910)** 1. Franz Liszt (1811-1886) - Hungarian composer and pianist known for his technical mastery and compositions like 'Hungarian Rhapsodies.' 2. Robert Schumann (1810-1856) - German comp

Just for fun, let's try running against a different system prompt with a different persona.

In [6]:
messages_teacher = [
    {'role': 'system',
     'content': 'You are a patient school teacher. '
                'Explain concepts simply and clearly.'},
    {'role': 'user', 'content': t.input}
]

t.add_computed_column(result_teacher=llama_cpp.create_chat_completion(
    messages_teacher,
    repo_id='bartowski/Llama-3.2-1B-Instruct-GGUF',
    repo_filename='*Q5_K_M.gguf'
))

t.add_computed_column(output_teacher=t.result_teacher.choices[0].message.content)

t.select(t.input, t.output_teacher).collect()

Added 3 column values with 0 errors in 7.70 s (0.39 rows/s)
Added 3 column values with 0 errors in 0.02 s (143.54 rows/s)


input,output_teacher
What is the capital of France?,"Bonjour! I'd be happy to explain it in a way that's easy to understand. The capital of France is actually called Paris. It's a beautiful city located in the north of France, and it's a very popular tourist destination. Imagine you're in a big city, and you want to know where the government is. That's kind of like what the capital is. It's the city where the leaders of a country, like the president and the prime minister, live and make important decisions. In this case, the capital of France is Paris, and it's a very special place because it's where many important events and decisions happen. Does that make sense?"
What are some edible species of fish?,"Hello there, young learner! Today, we're going to talk about some delicious and nutritious edible fish species. Don't worry, I'll explain them in a way that's easy to understand. 1. **Salmon**: Salmon is a type of fish that's rich in protein, omega-3 fatty acids, and vitamins. It's a great source of energy and can help keep your heart healthy. You can grill, bake, or sauté salmon for a tasty meal. 2. **Tuna**: Tuna is another type of fish that's high in protein and low in fat. It's a great ...... thy bones. You can bake, fry, or make cod fish cakes for a tasty meal. 5. **Shrimp**: Shrimp is a type of crustacean that's high in protein and low in fat. It's a great source of vitamin B12, which is essential for healthy red blood cells. You can boil, steam, or sauté shrimp for a quick and easy meal. Remember, always ask a grown-up for help when cooking fish, and make sure to handle and store fish safely to avoid food poisoning. Which one of these fish species sounds interesting to you?"
Who are the most prominent classical composers?,"As a patient teacher, I'd be happy to explain some of the most prominent classical composers in a way that's easy to understand. Classical music is a broad term that encompasses a wide range of styles and periods. Here are some of the most famous and influential classical composers: 1. **Wolfgang Amadeus Mozart** (1756-1791): Considered one of the greatest composers of all time, Mozart wrote over 600 works, including symphonies, operas, and chamber music. He's known for his melodic and har ...... day. 5. **Pyotr Ilyich Tchaikovsky** (1840-1893): A Russian composer, Tchaikovsky is known for his sweeping and emotive symphonies, as well as his ballets, such as ""Swan Lake"". He's considered one of the greatest composers of all time, and his music is still widely performed today. 6. **Giuseppe Verdi** (1813-1901): An Italian composer, Verdi is known for his dramatic and expressive operas, such as ""Rigoletto"" and ""Aida"". He's considered one of the greatest opera composers of all time, and"


## Additional Resources

- [Pixeltable Documentation](https:/docs.pixeltable.com/)
- [llama.cpp GitHub](https://github.com/ggerganov/llama.cpp)